***FCIM.FIA - Fundamentals of Artificial Intelligence***

> **Lab 2:** *Adversarial Search Algorithms* \\
> **Performed by:** *Bajenov Sevastian*, group *FAF-213* \\
> **Verified by:** Elena Graur, asist. univ.

## Imports and Utils

Create a virtual environment, install all the necessary dependencies so that you can run the notebook using your virtual environment as a kernel.

In [1]:
# pip install -r requirements.txt

In order to test Pacman's behavior use the following command:

`python pacman.py -p AgentClassNameAgent -l mediumClassic -a depth=3 --frameTime 0`

Where `AgentClassName` is the name of the class which inherits from the `MultiAgentSearchAgent` in the `multiAgents.py` file.

## Task 1

`Mini-max algorithm` is a recursive or backtracking algorithm which is used in decision-making and game theory. It provides an optimal move for the player assuming that opponent is also playing optimally. Mini-Max algorithm uses recursion to perform `adversarial` search through the game-tree. This Algorithm computes the minimax decision for the current state. It considers two players play the game, one is called MAX and other is called MIN. Both Players of the game are opponent of each other, where MAX will select the maximized value and MIN will select the minimized value.

In order to implement `Mini-max algorithm` it is necessary to define an `evaluation function` which should define criteria of maximizing or minimizing the score. The provided formula in the conditions was `Score = Distance_to_nearest_pellet - Distance_to_nearest_ghost` and it's ovious that it is wrong because it reaches its maximum when the pellets are too far anf the ghosts are too close to Pacman. That is why I changed it a bit, by also considering current game score: `Score = GameScore + Distance_to_nearest_ghost - Distance_to_nearest_pellet`. Below is the implementation of the default minimax algorithm presented.

In [2]:
# do not run this cell

class MinimaxAgent(MultiAgentSearchAgent):

    def __init__(self, evalFn='scoreEvaluationFunction', depth='2'):
        super().__init__(evalFn, depth)

    def getAction(self, gameState):
        bestAction = None
        bestScore = float('-inf')

        for action in gameState.getLegalActions(0):
            score = self.minimax(
                1, self.depth, gameState.generateSuccessor(0, action))

            if score > bestScore:
                bestScore = score
                bestAction = action

        return bestAction

    def minimax(self, agentIndex, depth, gameState):
        if depth == 0 or gameState.isWin() or gameState.isLose():
            score = self.evaluationFunction(gameState)
            return score

        numAgents = gameState.getNumAgents()
        nextAgent = (agentIndex + 1) % numAgents
        nextDepth = depth - 1 if nextAgent == 0 else depth

        legalActions = gameState.getLegalActions(agentIndex)

        if agentIndex == 0:
            score = max(self.minimax(nextAgent, nextDepth, gameState.generateSuccessor(
                agentIndex, action)) for action in legalActions)
        else:
            score = min(self.minimax(nextAgent, nextDepth, gameState.generateSuccessor(
                agentIndex, action)) for action in legalActions)

        return score

And here is the default evaluation function (manhattan_distance is being used as a heuristic parameter - distance between two points in the cartesian space):

In [ ]:
# do not run this cell

def scoreEvaluationFunction(currentGameState):

    def distanceToNearestPellet(gameState):
        pacmanPos = gameState.getPacmanPosition()
        food = gameState.getFood()
        minDistance = float('inf')

        for x in range(food.width):
            for y in range(food.height):
                if food[x][y]:
                    distance = manhattanDistance(pacmanPos, (x, y))

                    if distance < minDistance:
                        minDistance = distance

        return minDistance

    def distanceToNearestGhost(gameState):
        pacmanPos = gameState.getPacmanPosition()
        ghostPositions = gameState.getGhostPositions()
        minDistance = float('inf')

        for ghostPos in ghostPositions:
            distance = manhattanDistance(pacmanPos, ghostPos)

            if distance < minDistance:
                minDistance = distance

        return minDistance

    score = currentGameState.getScore()
    score -= distanceToNearestPellet(currentGameState)
    score += distanceToNearestGhost(currentGameState)

    return score

## Task 2

`Alpha-beta pruning` is a technique used to improve the efficiency of the minimax algorithm in game-playing AI. The technique is based on the observation that in many games, some branches of the game tree can be safely pruned (ignored) because they are guaranteed to be worse than other branches. 

The algorithm uses two parameters, `alpha` (best value for the maximizing player) and `beta` (best value for the minimizing player). The algorithm starts with alpha and beta having negative and positive infinity values, respectively. As the algorithm evaluates each node, it compares the score of the node with the current alpha and beta values. If the `current alpha value is > than the current beta value`, it means that the current branch is worse than a branch that has already been evaluated and can be safely pruned.

Code implementation of the `Alpha-beta pruning` is rather starightforward and is presented further. It uses the same evaluation function as the `Mini-max` algorithm. It is important to mention that the Alpha-beta pruning enhances default Mini-max because less number of game states is being verified, thus speeding up Pacman decision making.

In [ ]:
# do not run this cell

class AlphaBetaAgent(MultiAgentSearchAgent):

    def getAction(self, gameState):
        legalActions = gameState.getLegalActions(0)
        bestAction = None
        alpha = float('-inf')
        beta = float('inf')
        bestValue = float('-inf')

        for action in legalActions:
            value = self.alphaBeta(
                1, self.depth, gameState.generateSuccessor(0, action), alpha, beta)

            if value > bestValue:
                bestValue = value
                bestAction = action

            alpha = max(alpha, bestValue)

        return bestAction

    def alphaBeta(self, agentIndex, depth, gameState, alpha, beta):
        if depth == 0 or gameState.isWin() or gameState.isLose():
            return self.evaluationFunction(gameState)

        numAgents = gameState.getNumAgents()
        nextAgent = (agentIndex + 1) % numAgents
        nextDepth = depth - 1 if nextAgent == 0 else depth

        legalActions = gameState.getLegalActions(agentIndex)

        if agentIndex == 0:
            value = float('-inf')

            for action in legalActions:
                value = max(value, self.alphaBeta(
                    nextAgent, nextDepth, gameState.generateSuccessor(agentIndex, action), alpha, beta))
                alpha = max(alpha, value)

                if alpha >= beta:
                    break

            return value
        else:
            value = float('inf')

            for action in legalActions:
                value = min(value, self.alphaBeta(
                    nextAgent, nextDepth, gameState.generateSuccessor(agentIndex, action), alpha, beta))
                beta = min(beta, value)

                if alpha >= beta:
                    break

            return value

## Task 3

`Mini-max algorithm` is not ideal, especially the `evaluation function` which was provided before. Although it takes into consideration important parameters, many game states are still defined as equally-desirable by it (Pacman gets stuck and starts turning to the left and to the right without moving forward until a ghost appears). That is why I decided to modify evaluation function in several ways:

* consider the number of pellets in the current region, their density within a certain radius;

* make use of the ghost vulnerable state when Pacman eats an `ultra pellet` in its advantage;

* analyze the complexity of the maze around Pacman so that it has to make less directional decisions;

All of this parameters are being calculated within the score evaluation function by accessing current game state.

In [ ]:
# do not run this cell

def scoreEvaluationFunction(currentGameState):

    def distanceToNearestPellet(gameState):
        pacmanPos = gameState.getPacmanPosition()
        food = gameState.getFood()
        minDistance = float('inf')

        for x in range(food.width):
            for y in range(food.height):
                if food[x][y]:
                    distance = manhattanDistance(pacmanPos, (x, y))

                    if distance < minDistance:
                        minDistance = distance

        return minDistance

    def distanceToNearestGhost(gameState):
        pacmanPos = gameState.getPacmanPosition()
        ghostPositions = gameState.getGhostPositions()
        minDistance = float('inf')

        for ghostPos in ghostPositions:
            distance = manhattanDistance(pacmanPos, ghostPos)

            if distance < minDistance:
                minDistance = distance

        return minDistance

    def pelletNumberPerRegionParameter(gameState, regionSize=5):
        pacmanPos = gameState.getPacmanPosition()
        food = gameState.getFood()
        walls = gameState.getWalls()
        regionPelletCount = 0

        for dx in range(-regionSize, regionSize + 1):
            for dy in range(-regionSize, regionSize + 1):
                x, y = pacmanPos[0] + dx, pacmanPos[1] + dy

                if 0 <= x < walls.width and 0 <= y < walls.height and food[x][y]:
                    regionPelletCount += 1

        return regionPelletCount

    def mazeComplexityParameter(gameState, radius=4):
        pacmanPos = gameState.getPacmanPosition()
        walls = gameState.getWalls()
        complexity = 0

        for dx in range(-radius, radius + 1):
            for dy in range(-radius, radius + 1):
                if dx == 0 and dy == 0:
                    continue

                x, y = pacmanPos[0] + dx, pacmanPos[1] + dy

                if 0 <= x < walls.width and 0 <= y < walls.height and walls[x][y]:
                    complexity += 1

        return complexity

    def ghostVulnerabilityParameter(gameState):
        pacmanPos = gameState.getPacmanPosition()
        scaredGhosts = [ghostState for ghostState in gameState.getGhostStates(
        ) if ghostState.scaredTimer > 0]
        minScaredGhostDistance = float('inf')

        for ghostState in scaredGhosts:
            ghostPos = ghostState.getPosition()
            distance = manhattanDistance(pacmanPos, ghostPos)

            if distance < minScaredGhostDistance:
                minScaredGhostDistance = distance

        return 1 / (minScaredGhostDistance + 1) if minScaredGhostDistance != float('inf') else 0

    score = currentGameState.getScore()
    score -= distanceToNearestPellet(currentGameState)
    score += distanceToNearestGhost(currentGameState)
    score += 0.5 * pelletNumberPerRegionParameter(currentGameState)
    score += ghostVulnerabilityParameter(currentGameState)
    score -= 0.25 * mazeComplexityParameter(currentGameState)

    return score

It is worth noticing that the evaluation function is still not ideal. Although, Pacman doesn't get stuck when there are many pellets in the maze, but when there are two or three left it may avoid eating them because of the maze complexity. However, the results are much better than they were initially.

## Task 4

Further on with the `Mini-max algorithm` enhancement, it is possible to make the algorithm itself more advanced. For that I used two of the provided optimization techniques: `Progressive Deepening` and `Transposition Tables`.

`Progressive deepening` or `Iterative deepening search` is a state space/graph search strategy in which a depth-limited version of depth-first search is run repeatedly with increasing depth limits until the goal is found. The algorithm is optimal, meaning that it finds the shallowest goal. Since it visits all the nodes in the search tree down to depth `d` before visiting any nodes at depth `d + 1`, the cumulative order in which nodes are first visited is effectively the same as in breadth-first search. However, `Iterative deepening search` uses much less memory.

In my implementation the algorithm uses a range of depths to find the best score for the upcoming move and chooses the highest one obtained.

`Transposition tables` (inititally used in chess) let us optimize calculating the best move when we encounter situations where different plays results being in the same end state. Essentially once we get to one specific Pacman position we just store the result of the minimax calculation at that position in the transposition table. This means that later on if some other different list of moves arrives at the same position then it is not necessary to completely recalculate the minimax at that position because the operation was already done before and we can just look it up from the transposition table.

The implementation of these enhancements is rather straightforward and is shown in the code snippet below. After running the game with these changes the Pacman started making decisions quicker than in the default Mini-max.

In [ ]:
# do not run this cell

class MinimaxAgent(MultiAgentSearchAgent):

    def __init__(self, evalFn='scoreEvaluationFunction', depth='2'):
        super().__init__(evalFn, depth)
        self.transpositionTable = {}

    def getAction(self, gameState):
        bestAction = None
        bestScore = float('-inf')

        for currentDepth in range(1, self.depth + 1):
            self.transpositionTable.clear()

            for action in gameState.getLegalActions(0):
                score = self.minimax(
                    1, currentDepth, gameState.generateSuccessor(0, action))

                if score > bestScore:
                    bestScore = score
                    bestAction = action

        return bestAction

    def minimax(self, agentIndex, depth, gameState):
        stateKey = (agentIndex, depth, gameState)

        if stateKey in self.transpositionTable:
            return self.transpositionTable[stateKey]

        if depth == 0 or gameState.isWin() or gameState.isLose():
            score = self.evaluationFunction(gameState)
            self.transpositionTable[stateKey] = score
            return score

        numAgents = gameState.getNumAgents()
        nextAgent = (agentIndex + 1) % numAgents
        nextDepth = depth - 1 if nextAgent == 0 else depth

        legalActions = gameState.getLegalActions(agentIndex)

        if agentIndex == 0:
            score = max(self.minimax(nextAgent, nextDepth, gameState.generateSuccessor(
                agentIndex, action)) for action in legalActions)
        else:
            score = min(self.minimax(nextAgent, nextDepth, gameState.generateSuccessor(
                agentIndex, action)) for action in legalActions)

        self.transpositionTable[stateKey] = score

        return score

## Task 5

Another enhancement which can be made to the `Mini-max algorithm` is changing the path finding algorithm to the `A* search algorithm`. `A* Search` is an informed best-first search algorithm that efficiently determines the lowest cost path between any two nodes in a directed weighted graph with non-negative edge weights. Its evaluation function is used to determine which node to explore next. The evaluation function, `f(x)`, for the `A* search algorithm` is the following: `f(x) = g(x) + h(x)` where `g(x)` represents the cost to get to node `x` and `h(x)` represents the estimated cost to arrive at the goal node from node `x`. For the algorithm to generate the correct result, the evaluation function must be `admissible`, meaning that it never overestimates the cost to arrive at the goal node.

`h(x)` or `heuristic function` in my implementation returns the minimal distance to a pellet (from the list of pellets of the current game state) using `manhattan distance` function.

In [ ]:
# do not run this cell

def heuristic(gameState):
    pacmanPos = gameState.getPacmanPosition()
    food = gameState.getFood()
    minPelletDistance = float('inf')

    for x in range(food.width):
        for y in range(food.height):
            if food[x][y]:
                distance = manhattanDistance(pacmanPos, (x, y))

                if distance < minPelletDistance:
                    minPelletDistance = distance

    return minPelletDistance

The complete `A* Mini-max algorithm` is presented in the cell below. It is worth mentioning that the search optimization was applied to the default `Mini-max` that is why in terms of performance it is slower than the `Mini-max` with enhancements added in the previous task.

In [ ]:
# do not run this cell

class AStarMinimaxAgent(MultiAgentSearchAgent):

    def getAction(self, gameState):
        bestAction = None
        bestScore = float('-inf')

        for action in gameState.getLegalActions(0):
            score = self.aStarMinimax(
                1, self.depth, gameState.generateSuccessor(0, action))

            if score > bestScore:
                bestScore = score
                bestAction = action

        return bestAction

    def aStarMinimax(self, agentIndex, depth, gameState):
        pq = PriorityQueue()
        pq.push((agentIndex, depth, gameState, 0), 0)

        while not pq.isEmpty():
            agentIndex, depth, gameState, cost = pq.pop()

            if depth == 0 or gameState.isWin() or gameState.isLose():
                return self.evaluationFunction(gameState) - heuristic(gameState)

            numAgents = gameState.getNumAgents()
            nextAgent = (agentIndex + 1) % numAgents
            nextDepth = depth - 1 if nextAgent == 0 else depth

            legalActions = gameState.getLegalActions(agentIndex)

            if agentIndex == 0:
                score = float('-inf')
                for action in legalActions:
                    successor = gameState.generateSuccessor(agentIndex, action)
                    newCost = cost + \
                        self.evaluationFunction(
                            successor) - heuristic(successor)

                    pq.push((nextAgent, nextDepth, successor, newCost), newCost)
                    score = max(score, newCost)
            else:
                score = float('inf')
                for action in legalActions:
                    successor = gameState.generateSuccessor(agentIndex, action)
                    newCost = cost + \
                        self.evaluationFunction(
                            successor) - heuristic(successor)

                    pq.push((nextAgent, nextDepth, successor, newCost), newCost)
                    score = min(score, newCost)

        return score

## Task 6

In order to incorporate `A* search algorithm` into the `Alpha-beta pruning algorithm` it is only neccesary to introduce the `priority queue` to store the nodes which have to be explored for both variants - when the max agent (Pacman) is searching for an optimal move and when min agent (Ghost) performs the opposite action.

In [ ]:
# do not run this cell

class AStarAlphaBetaAgent(MultiAgentSearchAgent):

    def getAction(self, gameState):
        bestAction = None
        bestScore = float('-inf')
        alpha = float('-inf')
        beta = float('inf')

        pq = PriorityQueue()
        for action in gameState.getLegalActions(0):
            pq.push((action, 1, self.depth, gameState.generateSuccessor(
                0, action), alpha, beta), 0)

        while not pq.isEmpty():
            action, agentIndex, depth, state, alpha, beta = pq.pop()
            score = self.aStarAlphaBeta(agentIndex, depth, state, alpha, beta)

            if score > bestScore:
                bestScore = score
                bestAction = action

        return bestAction

    def aStarAlphaBeta(self, agentIndex, depth, gameState, alpha, beta):
        if depth == 0 or gameState.isWin() or gameState.isLose():
            return self.evaluationFunction(gameState) - heuristic(gameState)

        numAgents = gameState.getNumAgents()
        nextAgent = (agentIndex + 1) % numAgents
        nextDepth = depth - 1 if nextAgent == 0 else depth

        legalActions = gameState.getLegalActions(agentIndex)

        if agentIndex == 0:  # Pacman's turn to maximize
            value = float('-inf')
            pq = PriorityQueue()

            for action in legalActions:
                successor = gameState.generateSuccessor(agentIndex, action)

                newCost = self.evaluationFunction(
                    successor) - heuristic(successor)
                pq.push((action, nextAgent, nextDepth,
                        successor, alpha, beta), newCost)

            while not pq.isEmpty():
                _, nextAgent, nextDepth, successor, alpha, beta = pq.pop()
                value = max(value, self.aStarAlphaBeta(
                    nextAgent, nextDepth, successor, alpha, beta))

                alpha = max(alpha, value)

                if alpha >= beta:
                    break

            return value
        else:
            value = float('inf')
            pq = PriorityQueue()

            for action in legalActions:
                successor = gameState.generateSuccessor(agentIndex, action)

                newCost = self.evaluationFunction(
                    successor) - heuristic(successor)
                pq.push((action, nextAgent, nextDepth,
                        successor, alpha, beta), newCost)

            while not pq.isEmpty():
                _, nextAgent, nextDepth, successor, alpha, beta = pq.pop()
                value = min(value, self.aStarAlphaBeta(
                    nextAgent, nextDepth, successor, alpha, beta))

                beta = min(beta, value)

                if alpha >= beta:
                    break

            return value

## Conclusions:

In this laboratory work I primarily investigated Mini-max adversarial search algorithm and its modifications. By applying various enhancements I observed the changes in Pacman behavior during the game. I found out that Alpha-Beta pruning significantly increases Pacman's speed of making decisions. Progressive Deepening and Transposition Tables techniques led to the same result when being tested. At the same time by introducing additional parameters in the evaluation function I avoided situations when the Pacman got stuck in one position, however it still was not able to collect all the pellets to complete the game. Overall, adversarial search algorithms from this laboratory work are still not ideal, but they perform at a decent level in gaming problems.

## Acknowledgements

In this laboratory work I was assisted by Arteom Kalamaghin from FAF-211. During our discussion he explained the use of A* search algorithm in the Mini-max algorithm implementation.

## Bibliography:

1. https://www.javatpoint.com/mini-max-algorithm-in-ai
2. https://medium.com/@aaronbrennan.brennan/minimax-algorithm-and-alpha-beta-pruning-646beb01566c
3. https://en.wikipedia.org/wiki/Iterative_deepening_depth-first_search
4. https://stackoverflow.com/questions/20009796/transposition-tables
5. https://www.codecademy.com/resources/docs/ai/search-algorithms/a-star-search